In [1]:
import pandas as pd

import timeit
import os

from py2neo import Graph
import time 
import datetime


In [2]:
method = 'neo4j'

# if you use kubernetes, you can map the dicingDate to the arguments.
# dicingDate = os.getenv('DICING_DATE')
dicingDate = '2015-07-01'

In [ ]:
experiment2_file = '/data/experiment2.csv'

In [ ]:
result_columns = ['Analysis', 'Number Of Events', 'Loading Time', 'DFG Calculation Time', 'Total Time']

In [ ]:
def getConnection():
    return Graph('bolt://localhost:7687', auth=("neo4j", '1234'))

In [ ]:
def getDFG(conn, dicingDate):

    CMD="""
        match (t1:Attribute {{key:'concept_name'}})<--(e1:Event {{log_concept_name:'BPI2016'}})-[n]->(e2:Event {{log_concept_name:'BPI2016'}})-->(t2:Attribute {{key:'concept_name'}})
        where 
            date(e1.time_timestamp)<=date("{0}")
            and
            date(e2.time_timestamp)<=date("{0}")
        return t1.val as dfg_from, t2.val as dfg_to, count(n) as dfg_freq
    """.format(dicingDate)
    res = conn.run(CMD)
    return res.data()

In [ ]:
def getNumberOfEvents(conn, dicingDate):
    CMD="""
        match (e:Event {{log_concept_name:'BPI2016'}})
        where date(e.time_timestamp)<=date("{}")
        return count(e) as NumberOfEvents
    """.format(dicingDate)
    res = conn.run(CMD)
    return res.data()[0]['NumberOfEvents']

In [8]:
def getDFGFromNeo4j(result, method, dicingDate, result_columns):
    dicingDate = datetime.datetime.strptime(dicingDate, '%Y-%m-%d')
    
    conn = getConnection()

    queryDate = dicingDate
    queryDate = queryDate.strftime("%Y-%m-%d")

    numberOfEvents = getNumberOfEvents(conn, queryDate)

    time_start = timeit.default_timer()
    res = getDFG(conn, queryDate)
    time_stop = timeit.default_timer()

    Loading_Time = 0
    DFG_Calculation_Time = time_stop-time_start
    Total_Time = time_stop-time_start

    new_row = pd.DataFrame([[method, numberOfEvents, Loading_Time, DFG_Calculation_Time, Total_Time]]
                           , columns=result_columns)

    result = pd.concat([result, new_row])
    
    result = result.sort_values(by=result_columns)
    
    return result

In [9]:
if os.path.isfile(experiment2_file):
    result = pd.read_csv(experiment2_file)
else:
    result = pd.DataFrame(columns=result_columns)

In [10]:
result = getDFGFromNeo4j(result, method, dicingDate, result_columns)

In [12]:
result = result.sort_values(by=result_columns)
result.to_csv(experiment2_file, index = False)